# Eclipse Mapping with an Instrument/Stellar Baseline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry
from copy import deepcopy
import astropy.units as u
import astropy.constants as const
from astropy.table import Table
from corner import corner
import os
import hotspot_fitter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
starry.__version__

## Set up Forward Model

Use Earth map as an example map

In [ ]:
#map1 = starry.Map(4)
map1 = starry.Map(3)

map1.load("earth", sigma=0.05)
y_input = deepcopy(map1.y[1:])

fig, ax = plt.subplots()
map1.show(projection="rect",colorbar=True,ax=ax)
fig.savefig('plots/forward_model/forward_map_rect.png',dpi=150,bbox_inches='tight')


# Calculate the Brightness Temperature

### Note on instensity units

Check that I understand the units of the rectilinear map. Here is a note from the starry documentation on adding spots that possibly gives a clue about the normalization even though it is talking about spots

    Keep in mind that things are normalized in starry such that the disk-integrated flux (not the intensity!) of an unspotted body is unity. The default intensity of an unspotted map is 1.0 / np.pi everywhere (this ensures the integral over the unit disk is unity). So when you instantiate a map and add a spot of contrast c, you’ll see that the intensity at the center is actually (1 - c) / np.pi. This is expected behavior, since that’s a factor of 1 - c smaller than the baseline intensity.

### Relate the map to physical units

Calculate the rectilinear map and the brightness temperature map. Here we are using the following relation between flux and map and physical flux units.

I$_{physical}$ = I$_*$  I$_{map,starry} \pi$  R$_*^2$ / R$_p^2$,

where I$_{physical}$ is the planet intensity in physical units, I$_*$ is the average stellar intensity in physical units, I$_{map,starry}$ is the map evaluated by Starry that has an amplitude (`amp`) defined by the planet flux relative to the host star flux and R$_*^2$ / R$_p^2$ is the ratio of star and planet radii.

We can check this relation by calculating the flux of the planet if the intensity is uniform.

F = $\int I cos(\theta) d\Omega$, for polar angle $\theta$ and solid angle $\Omega$

For a uniform intensity sphere (See Rybicki & Lightman for the derivation of the flux of a uniform sphere in Chapter 1, page 8),

$F = \pi I \frac{R^2}{d^2}$

Calculating the intensity in physical units gives

$F_{physical} = \pi I_{physical} \frac{R_p^2}{d^2} = I_* \pi \frac{amp}{\pi} \pi \frac{R_p^2}{d^2} \frac{R_*^2}{R_p^2}. = \pi I_* amp \frac{R_*^2}{d^2} = F_* amp$

Here we have used I$_{map,starry} = \frac{amp}{\pi}$ from definition of the normalization above and the amplitude factor that starry multiplies the spherical harmonics by. So, this checks out here that the secondary eclipse depth $\left( \frac{F_p}{F_*}\right)^2$ is equal to the amplitude.




### Use the Physical Units to Determine Brightness Temperature

Using the I$_{physical}$ relation above, we can calculate Brightness Temperature. This gives

T$_{B,p} = \frac{C_2}{ln(arg)}$,

where T$_{B,p}$ is the planet brightness temperature, $C_2 = \frac{h c}{\lambda k_B}$, where h is Planck's constant, c is the speed of light, $\lambda$ is the wavelength, $k_B$ is Boltzmann's constant and $arg$ is

$arg = 1 + \frac{e^{C_2 / T_{*,B}} - 1}{I_{starry} C_0}$,

$T_{*,B}$ is the brightness temperature of the star and

$C_0 = \frac{\pi}{(R_p / R_* )^2}$

This is consistent with Rauscher et al. 2016 Equation 8

In [ ]:
def TB(intens,Tstar=5050. * u.K,wave=4.39 * u.micron,rp_over_rstar=0.1510):
    """
    Calculate brightness temperature from
    the intensity map, using information about the star and radius ratios
    """
    const_factor = (const.h * const.c) / (wave * const.k_B)
    
    intens_ratio = intens * np.pi / rp_over_rstar**2 # intensity ratio
    arg = 1. + (np.exp(const_factor/Tstar) - 1.)/intens_ratio
    
    return (const_factor / np.log(arg)).si

In [ ]:
res = 100

map1.amp = 0.001
map2D = map1.render(res=res,projection='rect')
lat = np.tile(np.linspace(-90,90,res),[res,1]).T
lon = np.tile(np.linspace(-180,180,res),[res,1])

lat_rad = lat * np.pi/180.
lon_rad = lon * np.pi/180.


TB_map = TB(map2D)

In [ ]:
fig, ax = plt.subplots()


rect_data = map1.render(res=res,projection='rect')
mapPlot = ax.imshow(TB_map,extent=[-180,180,-90,90],origin='lower',cmap='plasma')#,
                   #vmin=1000,vmax=1550)
fig.colorbar(mapPlot,label='Brightness Temperature (K)')
ax.set_xlabel("Longitude ($\degree$)")
ax.set_ylabel("Latitude ($\degree$)")

fig.savefig('plots/forward_model/forward_map_rect_TB_idealized_planet.pdf',dpi=150,bbox_inches='tight')


In [ ]:
t = Table()
t['y'] = map1.y
t.write('sim_data/y_original.csv',overwrite=True)

## Find the "hotspot"


In [ ]:
res = 100

map2D = map1.render(res=res,projection='rect')
lat = np.tile(np.linspace(-90,90,res),[res,1]).T
lon = np.tile(np.linspace(-180,180,res),[res,1])

In [ ]:
hf = hotspot_fitter.hotspot_fitter(map2D,lon,lat,ystart=63,yend=87,xstart=56,xend=76)

In [ ]:
hf.plot_fits()


In [ ]:
x_proj, y_proj = hf.get_projected_hostpot()


In [ ]:
hf.p_fit.x_mean.value, hf.p_fit.y_mean.value


In [ ]:
hf.p_fit

In [ ]:
hf.p_fit.bounds

In [ ]:
hf.check_cross_sections()

In [ ]:
plt.close(fig)
fig, ax = plt.subplots()

map1.amp = 0.001 * 1000. ## make ppt
map1.show(projection='ortho',ax=ax,colorbar=True)
ax.plot(x_proj,y_proj,'o',markersize=15)
colorbar = ax.images[-1].colorbar
colorbar.set_label('I (ppt)')
fig.savefig('plots/forward_model/forward_map.png',dpi=150,bbox_inches='tight')

map1.amp = 0.001 ## return to fractional units

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.5))

forward_map2D = map1.render(projection='rect')

lonpeak, latpeak = hf.return_peak()
pdata = ax.imshow(forward_map2D * 1000.,origin='lower',
                  extent=[-180,180,-90,90],
                 vmin=0,vmax=0.7,
                 cmap='plasma')

ax.plot(lonpeak,latpeak,'o',markersize=15)


divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

ax.set_xlabel("Longitude ($^\circ$)")
ax.set_ylabel("Latitude ($^\circ$)")
ax.set_xlim(-106.2,106.2)

fig.colorbar(pdata,label=r'I ($10^{-3}~ \mathrm{I}_*$)',cax=cax)
fig.savefig('plots/forward_model/forward_map_rect.pdf',dpi=150,bbox_inches='tight')

Make a short period hot Jupiter with this map

In [ ]:
M_star = 1.0
R_star = 1.0
inc=83
#inc=90.0
rp = 0.1
P_b = 1.0 ## days
log_amp = -3.
t0 = 0.0

M_planet = 0.0

prot_star = 1.0

x = np.linspace(0.5 * P_b - 0.1,0.5 * P_b + 0.1,2048)
#x = np.linspace(0.,P_b,4096)


Get the impact parameter

In [ ]:
norb = 2. * np.pi / (P_b * u.day)
a_orb = (const.G * (M_star * u.Msun) / norb**2)**(1./3.)
a_over_r_star = (a_orb/(R_star * u.Rsun)).si
b_impact = a_over_r_star * np.cos(inc * np.pi/180.)
b_impact

In [ ]:
a_orb.to(u.AU)

In [ ]:
a_over_r_star

In [ ]:
y_input

In [ ]:
y_table = Table()
y_labels1, y_values1 = [], []
for one_l in np.arange(3)+1:
    counter=1
    print("Y_{},m=[".format(one_l))
    for one_m in np.arange(-one_l,one_l+1):
        y_labels1.append("$Y_{{{},{}}}$".format(one_l,one_m))
        print("Y_{}_{} = {}".format(one_l,one_m,y_input[counter]))
        #print(y_input[counter])
        counter = counter + 1
    print("] \n")
y_table['label'] = y_labels1
y_table['value'] = np.round(y_input,4)
y_table['res'] = '\\nodata'
y_table.write('plots/forward_model/forward_mod.tex',overwrite=True)

Find the transit durations for contacts 1 to 4 and 2 to 23

In [ ]:
arg = np.sqrt((1. + rp/R_star)**2 - b_impact**2)/ (a_over_r_star * np.sin(inc * np.pi/180.))
Tdur_14 = (P_b / np.pi) * np.arcsin(arg)
Thalf_14 = (0.5 * Tdur_14).value

arg2 = np.sqrt((1. - rp/R_star)**2 - b_impact**2)/ (a_over_r_star * np.sin(inc * np.pi/180.))
Tdur_23 = (P_b / np.pi) * np.arcsin(arg2)
Thalf_23 = (0.5 * Tdur_23).value
Thalf_14,Thalf_23

In [ ]:
A = starry.Primary(starry.Map(ydeg=0, udeg=2, amp=1.0), m=M_star, r=R_star,
                   prot=prot_star )
b = starry.kepler.Secondary(map1,
                            m=0.0,r=rp,prot=P_b,porb=P_b,t0=t0,inc=inc)
b.map.amp = 10**log_amp

In [ ]:
b.theta0 = 180.0 + 0.0
sys = starry.System(A,b)

In [ ]:
def plot_model_w_data(x,model,data,yerr,ingressZoom=False):
    if ingressZoom == True:
        fig, axArr = plt.subplots(1,2,sharey=True,figsize=(9,4))
        outName = "forward_model_lc_zoom.pdf"
    else:
        fig, ax1 = plt.subplots(figsize=(9,4))
        axArr = [ax1]
        outName = "forward_model_lc.pdf"
        
    for i,ax in enumerate(axArr):
        ax.set_xlabel("Time (days)")
        if i==0:
            ax.set_ylabel("Normalized Flux")
        ax.errorbar(x,ysim,yerr=yerr,fmt='.',zorder=0)
        ax.plot(x,flux_input,zorder=2)

        
        if ingressZoom == True:
            #ax.set_xlim(0.5 * P_b - Thalf_14,0.5 * P_b - Thalf_23)
            t_ing = Thalf_14 - Thalf_23
            t1 = Thalf_14 + t_ing * 0.2
            t2 = Thalf_23 - t_ing * 0.2
            #t1, t2 = 0.044, 0.0325
            if i==0:
                
                ax.set_xlim(0.5 * P_b - t1,0.5 * P_b -t2)
            else:
                ax.set_xlim(0.5 * P_b + t2,0.5 * P_b + t1)
    savePath = os.path.join('plots','forward_model',outName)
    fig.savefig(savePath)
                
flux_input = deepcopy(sys.flux(x))

np.random.seed(0)
yerr = np.ones_like(x) * 15e-6
ysim = flux_input + np.random.randn(len(x)) * yerr

plot_model_w_data(x,flux_input,ysim,yerr)




In [ ]:
plot_model_w_data(x,flux_input,ysim,yerr,ingressZoom=True)

In [ ]:
def compute_info(A):
    """Compute some information about the null space of the design matrix A."""
    # Get the Fisher information & compute its rank
    I = A.T.dot(A)
    R = np.linalg.matrix_rank(I)

    # Number of coefficientss
    C = I.shape[0]

    # Size of null space
    N = C - R

    # Fractional size of null space
    F = N / C

    # Show it graphically
    fig, ax = plt.subplots(figsize=(6, 0.3))
    ax.set_xlim(0, 1)
    ax.axis("off")
    ax.axvspan(0, 1 - F, color="C0")
    ax.axvspan(1 - F, 1, color="red")
    ax.annotate(
        "{}/{}".format(R, C),
        color="C0",
        fontsize=10,
        xy=(-0.025, 0.5),
        xycoords="axes fraction",
        va="center",
        ha="right",
    )
    ax.annotate(
        "{:.0f}%".format(100 * F),
        color="w",
        fontsize=10,
        xy=(1 - 0.5 * F, 0.5),
        xycoords="axes fraction",
        va="center",
        ha="right",
    )

Check the size of the nullspace

In [ ]:
A = sys.design_matrix(x)
compute_info(A)

In [ ]:
for i in np.arange(6):#sec.map.Ny):
    plt.plot(A[:,i] - 0.6 * i)

For degree=4 there's a pretty small nullspace, but still some coefficints are unconstrained. For degree=3 I don't see any nullspace which is pretty cool!

# Solve the Linear System

In [ ]:
# Prior on primary
# pri_mu = np.zeros(sys.primary.map.Ny)
# pri_mu[0] = 1.0
# pri_L = np.zeros(pri.map.Ny)
# pri_L[0] = 1e-2
# pri_L[1:] = 1e-2
# pri.map.set_prior(mu=pri_mu, L=pri_L)

# Prior on the planet = secondary
sec = sys.secondaries[0]
sec_mu = np.zeros(sec.map.Ny)
sec_mu[0] = 1e-3
## sec_mu[1:] = y_input * sec_mu[0]## what if we cheat at start them at the correct values? Just to check 
## that it can recover
sec_L = np.zeros(sec.map.Ny)
sec_L[0] = (0.2 * sec_mu[0])**2 ## covariance is squared
#sec_L[1:] = (1e-8)**2
sec_L[1:] = (1.0 * sec_mu[0])**2
sec.map.set_prior(mu=sec_mu, L=sec_L)

In [ ]:
sys.set_data(ysim, C=yerr**2)

In [ ]:
mu, cho_cov = sys.solve(t=x)

In [ ]:
sec.map.amp = mu[0]
sec.map[1:, :] = mu[1:] / sec.map.amp
sys.secondaries[0].map = sec.map

plot_model_w_data(x,sys.flux(x),ysim,yerr,ingressZoom=False)

In [ ]:
plot_model_w_data(x,sys.flux(x),ysim,yerr,ingressZoom=True)

### Check the amplitudes
The solution given gives the Cholesky decomposition of the covariance matrix (not the actual covariance matrix). I think this is how you get the covariance matrix. You can also skip down to the corner plot, where it looks good.

In [ ]:
cov = np.dot(cho_cov,cho_cov.T)
np.sqrt(np.diag(cov))

Compare the amplitudes and spherical harmonics

In [ ]:
print(10**log_amp, mu[0], np.sqrt(cov[0,0]))

mu[1:]/mu[0]

coeff_ind = np.arange(len(y_input)) + 1
coeff_mean = mu[1:]/mu[0]
coeff_err = np.sqrt(np.diag(cov))/mu[0]

labels = [r"$Y_{%d,%d}$" % (l, m)
    for l in range(1, sec.map.ydeg + 1)
    for m in range(-l, l + 1)
]




fig, ax = plt.subplots(figsize=(12,6))
ax.plot(coeff_ind,y_input,label='Input Forward Model')
ax.errorbar(coeff_ind,coeff_mean,yerr=coeff_err[1:],label='Posterior Mean')
ax.set_xlabel("Coefficient index")
ax.set_ylabel("$C_{l}^m$")
ax.legend()

for ind,oneLabel in enumerate(labels):
    ax.text(coeff_ind[ind] - 0.3,-0.9 + np.mod(ind,2) * 0.2,oneLabel)



### Original Input Map - Dayside

In [ ]:
sec = sys.secondaries[0]
sec.map.amp = 10**log_amp
sec.map.y[1:] = y_input
#sec.map.show(projection='rect')
sec.map.show(projection='ortho',theta=0.0)
#sec.map.show(theta=180.0)

In [ ]:
## Confirm that this is the dayside by visualizing the whole system
#sys.show(0.46 * P_b,figsize=(6,6))

## Mean Map

In [ ]:
# mean values
sec = sys.secondaries[0]

# sec.map.amp = mu[0]
# ## The tutorial notebook divides mu by amp, but it didn't look correct
# sec.map[1:, :] = mu[1 : sec.map.Ny]  #/ sec.map.amp

sec.map.amp = mu[0]
sec.map[1:, :] = mu[1:] / sec.map.amp

sec.map.show(projection="ortho")

In [ ]:
sec.map.y[1:], y_input

Check the residuals

In [ ]:
sys.secondaries[0].map = sec.map
yfit = sys.flux(x)
resid = yfit - ysim

fig, (ax,ax2) = plt.subplots(2,sharex=True)
ax.plot(x,yfit)
ax.errorbar(x,ysim,alpha=0.5)
ax2.plot(x,resid)

Compare a few draws with the truth

In [ ]:
np.random.seed(4)
for i in np.arange(5):
    sys.draw()
    sys.secondaries[0].map.show(projection='ortho')

In [ ]:
truths = np.append(10**log_amp,y_input[:8])

nsamples = 10000
u = np.random.randn(len(mu), nsamples)
samples = mu.reshape(1, -1) + np.dot(cho_cov, u).T

# De-weight the samples to get
# samples of the actual Ylm coeffs
samps = np.array(samples[:, :9])
samps[:, 1:] /= samps[:, 0].reshape(-1, 1)

In [ ]:
fig, ax = plt.subplots(9, 9, figsize=(12, 12))
labels = [r"$\alpha$"] + [
    r"$Y_{%d,%d}$" % (l, m)
    for l in range(1, sec.map.ydeg + 1)
    for m in range(-l, l + 1)
]

corner(samps, fig=fig, labels=labels,truths=truths)
for axis in ax.flatten():
    axis.xaxis.set_tick_params(labelsize=6)
    axis.yaxis.set_tick_params(labelsize=6)
    axis.xaxis.label.set_size(12)
    axis.yaxis.label.set_size(12)
    axis.xaxis.set_label_coords(0.5, -0.6)
    axis.yaxis.set_label_coords(-0.6, 0.5)

## Now let's add in a baseline trend

In [ ]:
x_norm = (x - np.mean(x))/(0.5 * (np.max(x) - np.min(x)))
c = np.array([2e-4,2e-4,3e-4,1.]) ## backwards order c_0 x^n + c_(1) x^(n-1) + ... c_(n-2) x^2 + c_(n-1) x + 1
baseline = np.polyval(c,x_norm)

In [ ]:
bsim = ysim * baseline

In [ ]:
plt.plot(x,bsim)
plt.xlabel('Time (JD)')
plt.ylabel("Normalized Flux")

Save the baseline

In [ ]:
outTable = Table()
outTable['Time (days)'] = x
outTable['Flux'] = bsim
outTable['Flux err'] = yerr
outTable['Flux before Baseline'] = ysim
outTable['Baseline'] = baseline

meta1 = {'Amplitude':10**log_amp,
         'Period':P_b,
         'Period_units': 'days',
         'M_star': M_star,
         'M_star_units': 'Msun',
         'R_star': R_star,
         'M_planet': M_planet,
         'M_planet_units': 'Msun',
         'inc': inc,
         'inc_units': 'deg',
         'rp': rp,
         'rp_units': 'Rsun',
         't0': t0,
         'y_input': list(y_input),
         'baseline_c': list(c),
         'dur_14': Thalf_14 * 2.,
         'dur_23': Thalf_23 * 2.,
         'b_impact': b_impact,
         'prot_star': prot_star}
outTable.meta = meta1
outTable.write('sim_data/sim_data_baseline.ecsv',overwrite=True)

Side notes: what if we added a baseline rather than multiplied by one?

In [ ]:
c2 = deepcopy(c)
c2[-1] = 0.0
bsim2 = ysim + np.polyval(c2,x_norm)


#plt.plot(x,bsim)
#plt.plot(x, bsim2)
plt.plot(x,bsim-bsim2)

Save the noiseless and noisy model

In [ ]:
fig, ax = plt.subplots(1,sharex=True)
ax.plot(x,flux_input,label='Noiseless Model',zorder=2,linewidth=2,color='black')
## colors are supposed to be color-blind-friendly
ax.plot(x,ysim,'.',label='Simulated Data: Flat Baseline',zorder=1,color='#FF9B54') #Sandy Brown
ax.plot(x,bsim,'.',label='Simulated Data: Baseline Trend',zorder=1,color='#720026') #Claret
ax.set_ylabel("Normalized Flux")
ax.set_xlabel("Time (days)")
ax.legend()
fig.savefig('plots/forward_model/forward_model_w_baseline.pdf',bbox_inches='tight')